# Brooder and Finisher Optimizer for Turkey Logistics

## Objective and Prerequisites

A large turkey grower must transport young turkeys from barns called brooders to barns
called finishers. There are 10 brooders and 62 finishers within a ~50 mile radius of the
processing facility. Currently, the company transports birds from the brooder to the first
available (or one of a few available) finisher even if they must transport to the furthest finisher
from the brooder that is being emptied that week. Therefore, if a linear program (LP) could be
formulated seeking to minimize the distance traveled the company may benefit by reducing
transportation costs and bird mortality.

**Download the Repository** <br /> 
You can download the repository containing this and other examples by clicking [here](https://github.com/bazylhorsey/livestock-logistic-optimizer). 

**Gurobi License** <br />
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an [evaluation license](https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-MFG-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_food-manufacturing_2_COM_EVAL_GITHUB_&utm_term=food-manufacturing-problem&utm_content=C_JPM) as a *commercial user*, or download a [free license](https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-MFG-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_food-manufacturing_2_ACADEMIC_EVAL_GITHUB_&utm_term=food-manufacturing-problem&utm_content=C_JPM) as an *academic user*.

---
## Problem Description

The objective function of the problem will be to minimize the kilometers driven between
brooders and finishers over one year
Before constraints can be written, a proper understanding of the turkey growing operation
must be acquired. The birds (called “poults” at this point in their life) are purchased and
delivered at one day old from nearby hatcheries and spend roughly the first 35 days of their lives
living in barns called brooders. A brooder can typically hold anywhere between 100,000 and
170,000 poults. After 35 days have elapsed and the poults have nearly outgrown the brooder barn
they are transported on trucks (approximately 3,000 at a time) to farms called finishers. A
finisher is comprised of two to five barns with each barn having a capacity of 11,000 birds. After
brooder barns are emptied they are cleaned and the bedding (of sawdust) is changed; this process
typically takes about 19 days. Once birds have been emptied into a finisher they grow for an
additional 105 days before being loading onto trucks and are taken to the plant. After a finisher is
emptied it is cleaned over a period of roughly 20 days and prepared for the next flock of birds.
Several years ago there was an international pandemic of avian flu which culled the birds
of many poultry operations in the region and around the world. Since that avian flu outbreak
poultry growers have enacted strict biosecurity measures to ensure the health and quality of their
birds. One such measure is to mandate that no birds of different age groups are mixed together.
For practical purposes, “same age” is taken to mean that they are born within one week apart.

There are a number of factors which might be relevant in the “real world” but cannot be
considered in this model due the difficulty involved in representing them mathematically. Some
of these factors include:

- Bird mortality: typically mortality is roughly 3% in brooders and 12% in finishers. Bird
mortality could be described as stochastic and is dependent on travel time, weather, bird
stress, and overall bird health

- Birds of the same age in different brooders: if birds of the same age are raised in different
brooders it is permissible for them to be placed at the same finisher

- Inexactness of capacity: while brooders and finishers have certain capacities it is unusual
for them to contain that exact amount of birds. For example, sometimes a finisher barn
may be slightly overfilled with 11,500 birds or underfilled with only 10,000 birds.
Additionally, sometimes brooders will contain fewer than their capacity allows due to
mortality or the hatchery slightly underdelivering on an order

- Inexactness of growing and cleaning times: often birds may stay in a brooder or finisher
for longer than 35 or 105 days. For example, if the processing facility is unable to keep
up with the production schedule (which was determine 1-2 years in advance) due to
insufficient labor, birds may end up staying in a finisher for an additional 30 days or
more. Additionally, a farmer may take shorter or longer than 19 or 20 days to clean their
barn after it is emptied

While important to consider, these points will not be factored into this particular model
formulation. However, should the company wish to run a full-scale model if this preliminary one
proves successful some of these may be wise to consider.


For the purposes of this model there are three main types of constraints and
considerations:
- Turkey life cycle
  - Brooders
    - Each brooder is occupied for 35 days prior to being emptied
    - Each brooder is cleaned over a period of 19 days after it is emptied
    - Therefore, it could be said that the average brooder is emptied (365/54) = 6.759 times per year
  - Finishers
    - Each finisher is occupied for 105 days prior to being emptied
    - Each finisher is cleaned over a period of 20 days after it is emptied
    - Therefore, it could be said that the average finisher receives (365/125) = 2.92 flocks of birds from brooders per year
- Farm and truck capacity
  - Brooders
    - Each brooder has a certain capacity of between 110,000 and 170,000
    - All brooders must be completely emptied after 35 days
  - Finishers
    - Each finisher may contain anywhere between 2 and 5 barns
    - Each barn has a capacity of 11,000 birds
    - Therefore, each finisher (farm) has a capacity of 22,000, 33,000, 44,000, or 55,000
    - The capacity of birds a finisher/ a group of finishers may receive from any given brooder over a year must be equal to or exceed the amount of birds that are actually sent to it
  - Trucks
    - Each truck can carry only 3,000 birds at a time
    - Therefore, the number of truckloads required to transport birds from brooders to finishers are 8, 11, 15, and 19 for finishers of capacity 22,000, 33,000, 44,000, 55,000 respectively (truckloads are rounded up because there can be no fraction of a load)
- Biosecurity
  - Birds may only be in the same brooder or finisher if they are born no more than a week apart
  - (For this scaled-down problem) Each finisher may be assigned only one brooder. i.e. even if birds in different brooders are of the same age they still cannot be grouped together in the same finisher

In [28]:
import pandas as pd

df = pd.read_csv("data/large/finisher.csv")
df


,Capacity,Distance Coefficient
0,55000,19
1,55000,19
2,55000,19
3,55000,19
4,55000,19
...,...,...
61,44000,15
62,33000,11
63,33000,11
64,55000,19


In [29]:
df = pd.read_csv("data/large/brooder.csv")
df


,Capacity
0,187000
1,182000
2,143000
3,165000
4,154000
5,165000
6,110000
7,110000
8,110000
9,182000


In [30]:
df = pd.read_csv("data/large/distance.csv", header=None)
df

,0,1,2,3,4,5,6,7,8,9
0,20.92142,27.35878,25.74944,22.53076,19.31208,17.70274,30.57746,30.57746,25.74944,33.79614
1,65.98294,33.79614,57.93624,41.84284,46.67086,78.85766,78.85766,78.85766,86.90436,65.98294
2,45.06152,43.45218,40.23350,57.93624,46.67086,82.07634,98.16974,98.16974,88.51370,33.79614
3,72.42030,72.42030,67.59228,85.29502,74.02964,111.04446,125.52852,125.52852,117.48182,65.98294
4,57.93624,59.54558,62.76426,43.45218,54.71756,24.14010,14.48406,14.48406,35.40548,67.59228
...,...,...,...,...,...,...,...,...,...,...
61,51.49888,20.92142,45.06152,27.35878,32.18680,51.49888,49.88954,49.88954,61.15492,53.10822
62,19.31208,32.18680,17.70274,33.79614,22.53076,59.54558,74.02964,74.02964,67.59228,9.65604
63,33.79614,9.65604,25.74944,24.14010,16.09340,49.88954,65.98294,65.98294,57.93624,37.01482
64,35.40548,35.40548,32.18680,48.28020,37.01482,74.02964,86.90436,86.90436,80.46700,24.14010


---
## Model Formulation

### Sets and Indices

$f \in \text{F}=\{\text{f}: \text{f} \in \mathbb{W}  ,  0 \le f < |F|\}$: Serial set of finishers.

$b \in \text{B}=\{\text{b}: \text{b} \in \mathbb{W}  ,  0 \le b < |B|\}$: Serial set of brooders.


### Parameters

$\text{c}_f \in \mathbb{N}^+$: Capacity of finisher $f \in Finisher$. Provided with coeffecient $2.944$ based on birds dispursed per year.

$\text{c}_b \in \mathbb{N}^+$: Capacity of brooder $b \in Brooder$. Provided with coeffecient $6.715$ based on birds recieved per year.

$\text{d}_{b,f} \in \mathbb{R}^+$: Distance between brooder $b$ and finisher $f$ (in kilometers).

$\text{k}_{f} \in \mathbb{N}^+$: Coeffecient of distance based on number of 3,000-bird truck trips to fill a finisher.

### Decision Variables

$\text{x}_{b, f} \in \{0,1\}$: 1 if finisher $f$ is to recieve birds from brooder $b$; 0 otherwise.


### Objective Function

- **Distance**: Minimize the total distance (in kilometers) to deliver brooder stock to finishers.

\begin{equation}
\text{Minimize}\quad Z = \sum_{f \in \text{F}}\sum_{b \in \text{B}}(\text{k}_{f}*\text{d}_{b,f}*\text{x}_{b, f})
\tag{0}
\end{equation}

### Constraints
- **Capacity**: Assert the capicity load of a brooder can fit into the recieved cumulative capacity of its finishers and that if it is scheduled to recieve.

\begin{equation}
2c_b \le \sum_{f \in F}c_{f}x_{b,f} \quad \forall  b \in B
\tag{1}
\end{equation}

- **Single Brooder Assignment**: Assert that each finisher $f$ is assigned exactly one brooder.

\begin{equation}
\sum_{b \in B}x_{b,f}=1 \quad \forall f \in F
\tag{2}
\end{equation}

---
## Python Implementation
We import the Gurobi Python Module and other Python libraries.

In [31]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB


## Preprocessing
We define all the input data of the model using the CSV.

In [32]:
# Parameters

distances_frame = pd.read_csv("data/large/distance.csv", header=None)
brooders_frame = pd.read_csv("data/large/brooder.csv")
finishers_frame = pd.read_csv("data/large/finisher.csv")

brooder_capacity = []
for index, row in brooders_frame.iterrows():
    brooder_capacity.append(row[0])

finisher_capacity = []
finisher_distance_k = []
for index, row in finishers_frame.iterrows():
    finisher_capacity.append(row[0])
    finisher_distance_k.append(row[1])

distances = {}
for i, row in distances_frame.iterrows():
    for j in range(0, len(row)):
        distances[(j, i)] = row[j]
edge, distance = gp.multidict(distances)

NUM_OF_BROODERS = len(brooder_capacity)
NUM_OF_FINISHERS = len(finisher_capacity)
BK = 2


## Model Deployment

We now determine the MIP model for the brooder allocation problem, by defining the decision variables, constraints, and objective function. Next, we start the optimization process and Gurobi finds the plan to build facilities that minimizes total costs.


In [33]:
# MIP  model formulation

m = gp.Model("brooder_allocation")
x = m.addVars(NUM_OF_BROODERS, NUM_OF_FINISHERS, vtype=GRB.BINARY, name="X")

Explain what constraints do

In [34]:
m.addConstrs(
    (
        gp.quicksum(x[b, f] for b in range(NUM_OF_BROODERS)) == 1
        for f in range(NUM_OF_FINISHERS)
    ),
    name="Assignment",
)


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

More on constraints?

In [35]:
m.addConstrs(
    (
        gp.quicksum(
            finisher_capacity[f] * x[b, f] for f in range(NUM_OF_FINISHERS)
        )
        >= BK * brooder_capacity[b]
    )
    for b in range(NUM_OF_BROODERS)
)


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>}

The objective is to minimize overall distance traveled.

In [36]:
m.setObjective(
    (
        gp.quicksum(
            distances[b, f] * finisher_distance_k[f] * x[b, f]
            for b in range(NUM_OF_BROODERS)
            for f in range(NUM_OF_FINISHERS)
        )
    ),
    GRB.MINIMIZE,
)


Next, we start the optimization and Gurobi finds the optimal solution.

In [37]:
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 1600 Six-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 76 rows, 660 columns and 1320 nonzeros
Model fingerprint: 0xb62cf620
Variable types: 0 continuous, 660 integer (660 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+04]
  Objective range  [2e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+05]
Presolve time: 0.00s
Presolved: 76 rows, 660 columns, 1320 nonzeros
Variable types: 0 continuous, 660 integer (660 binary)
Found heuristic solution: objective 47350.001480
Found heuristic solution: objective 47100.553780

Root relaxation: objective 2.471536e+04, 132 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/N

---
## Analysis

Analysis and thoughts

### Travel Plan
Defines the amount of kilometers driven if the model is used.

In [38]:
vals = m.getAttr("x", x)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] == 1.0)
df = pd.DataFrame(np.zeros((NUM_OF_FINISHERS, NUM_OF_BROODERS)))
for i in selected:
    df.at[i[1],i[0]]=1
df

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
61,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
63,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [39]:


df = pd.DataFrame(np.zeros((NUM_OF_FINISHERS, NUM_OF_BROODERS)))
df.replace(0, np.nan, inplace=True)
for i in selected:
    df.at[i[1],i[0]]=distances.get(i,j) * finisher_distance_k[j]
df

,0,1,2,3,4,5,6,7,8,9
0,NaN,NaN,NaN,NaN,NaN,336.35206,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,795.01396,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,642.12666
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1253.67586
4,NaN,NaN,NaN,NaN,NaN,NaN,275.19714,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
61,NaN,NaN,NaN,519.81682,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,183.46476
63,NaN,183.46476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,458.66190


### Chosen Route Plan

Defines which route a brooder is allowed to deliver to within the constraints.

In [40]:
print(df.sum().sum())



27856.066059999997


Note: If you want to write your solution to a file, rather than print it to the terminal, you can use the model.write() command. An example implementation is:

`m.write("brooder-allocation-output.sol")`

---
## References

Copyright © Griffin Wilson and Bazyl Horsey